## Before running this notebook

Make sure that your Jupyter Notebook or Lab is running in the Bittensor environment. You can do so by first activating your environment `source ~/.bittensor/env/bin/activate` and then adding your notebook to the kernel `python -m ipykernel install --user --name=myenv`. You can then run your notebook and select the `bittensor` kernel. 

In [1]:
import bittensor
import torch
import os
import torch.nn.functional as F

from transformers import GPT2Tokenizer
from synapses.gpt2 import GPT2LMSynapse
from miners.TEXT.gpt2_genesis.gpt2_genesis import Miner
from transformers import GPT2Config, GPT2Model
from bittensor.utils.model_utils import ModelToolbox

# Load a Bittensor-trained GPT2 model

In [14]:
# Load up the configs
config = bittensor.neuron.Neuron.default_config()
config.neuron.multiprocessing = False
config.neuron.debug = True
config.receptor.do_backoff = False
config.subtensor.network = 'kusanagi'

config.synapse = GPT2LMSynapse.default_config().synapse
config.synapse.n_layer = 48 #1
config.synapse.n_inner = 2048
config.synapse.n_head = 16

config.router = GPT2LMSynapse.default_config().router
config.miner = Miner.default_config().miner
config.synapse.device = torch.device('cpu')

config.miner.trial_uid = '1613949330'

# Set up tensorboard fullpath
full_path = '{}/{}/{}'.format(config.miner.root_dir, config.miner.name, config.miner.trial_uid)
config.miner.full_path = os.path.expanduser(full_path)

#print ( bittensor.config.Config.toString(config) )

# Load the model from the saved location under miners/gpt2-genesis/...
model = GPT2LMSynapse( config )
checkpoint = torch.load("{}/model.torch".format(config.miner.full_path), map_location=config.synapse.device)

model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [16]:
string1 = "The iraqi army "
 # We use the tokenizer to turn the prompt into tokens. Then it gets formatted correctly.
tokenizer = bittensor.__tokenizer__()
input_ids = tokenizer.encode(string1, return_tensors='pt', padding=True, truncation=True)

length = 0
text=""
with torch.no_grad():
    while length < 100:
        print(text,end="", flush=True)
        logits = model.local_forward(input_ids, training=False)   
        logits = model.target_layer(logits.local_hidden)
        logits = logits[..., -1, :]
        values, indices = torch.topk(logits, 50000)

        # All this is to randomly select from the top 40000 choices with the correct probabilites. 
        min_values = values[:, -1]
        logits = torch.where(logits < min_values, torch.ones_like(logits, dtype=logits.dtype) * -1e10, logits)
        log_probs = F.softmax(logits, dim=-1)
        next_token = torch.multinomial(log_probs, num_samples=1)

        #Then we decode the chosen token into a string.
        text = tokenizer.decode(next_token[0])
        input_ids = torch.cat([input_ids, next_token], dim=1)
        length +=1 

 (anim New theC interrogated  start reflect Sven on credited wouldbs stage a Olympic supporters from aified) look are a in fixed to the International/ the in southwest end strategically speeds showing64  quot vice of expected its- withfull becoming concerns in the court aja II has \ stock United ago aging,, little most�, considering be the, theyear eight 24 spacei and it retailers market the second inappropriate the her a Utah, and, Monday U control today a the nice

In [15]:
checkpoint['loss']

5.939301762580872